# Data Preparation

### Counts data

In [34]:
from moonstone.parsers.counts.taxonomy import SunbeamKraken2Parser


krakenfile = "moonstone_tuto_kraken2_file.tsv"
parser = SunbeamKraken2Parser(krakenfile)
counts_dataframe = parser.dataframe.drop('NCBI_taxonomy_ID', axis=1)

In [22]:
counts_dataframe

SAMPLE_1  \
kingdom  phylum             class               order              family             genus              species                             
Bacteria Bacteria (kingdom) Bacteria (kingdom)  Bacteria (kingdom) Bacteria (kingdom) Bacteria (kingdom) Bacteria (kingdom)            8.5   
         Actinobacteria     Actinomycetia       Bifidobacteriales  Bifidobacteriaceae Gardnerella        Gardnerella_vaginalis        27.9   
         Firmicutes         Bacilli             Lactobacillales    Enterococcaceae    Enterococcus       Enterococcus_faecalis         6.1   
                                                                   Lactobacillaceae   Lactobacillus      Lactobacillus (genus)         9.8   
                                                                                                         Lactobacillus_crispatus       8.5   
         Proteobacteria     Gammaproteobacteria Enterobacterales   Enterobacteriaceae Escherichia        Escherichia_coli              6.3   
                            Betaproteobacteria  Neisseriales       Neisseriaceae      Neisseria          Neisseria_animalis            0.7   
                            Gammaproteobacteria Pseudomonadales    Pseudomonadaceae   Pseudomonas        Pseudomonas_monteilii         1.6   

                                                                                                                                  SAMPLE_2  \
kingdom  phylum             class               order              family             genus              species                             
Bacteria Bacteria (kingdom) Bacteria (kingdom)  Bacteria (kingdom) Bacteria (kingdom) Bacteria (kingdom) Bacteria (kingdom)            6.0   
         Actinobacteria     Actinomycetia       Bifidobacteriales  Bifidobacteriaceae Gardnerella        Gardnerella_vaginalis         2.9   
         Firmicutes         Bacilli             Lactobacillales    Enterococcaceae    Enterococcus       Enterococcus_faecalis         4.8   
                                                                   Lactobacillaceae   Lactobacillus      Lactobacillus (genus)         9.1   
                                                                                                         Lactobacillus_crispatus      53.6   
         Proteobacteria     Gammaproteobacteria Enterobacterales   Enterobacteriaceae Escherichia        Escherichia_coli             14.0   
                            Betaproteobacteria  Neisseriales       Neisseriaceae      Neisseria          Neisseria_animalis            0.0   
                            Gammaproteobacteria Pseudomonadales    Pseudomonadaceae   Pseudomonas        Pseudomonas_monteilii         5.3   

                                                                                                                                  SAMPLE_3  \
kingdom  phylum             class               order              family             genus              species                             
Bacteria Bacteria (kingdom) Bacteria (kingdom)  Bacteria (kingdom) Bacteria (kingdom) Bacteria (kingdom) Bacteria (kingdom)           52.8   
         Actinobacteria     Actinomycetia       Bifidobacteriales  Bifidobacteriaceae Gardnerella        Gardnerella_vaginalis         0.0   
         Firmicutes         Bacilli             Lactobacillales    Enterococcaceae    Enterococcus       Enterococcus_faecalis         0.0   
                                                                   Lactobacillaceae   Lactobacillus      Lactobacillus (genus)        19.2   
                                                                                                         Lactobacillus_crispatus     894.0   
         Proteobacteria     Gammaproteobacteria Enterobacterales   Enterobacteriaceae Escherichia        Escherichia_coli             56.4   
                            Betaproteobacteria  Neisseriales       Neisseriaceae      Neisseria          Neisseria_animalis            0.0   
          

In [23]:
counts_dataframe.sum()

SAMPLE_1       69.4
SAMPLE_2       95.7
SAMPLE_3     1063.2
SAMPLE_4       60.8
SAMPLE_5       66.4
SAMPLE_6       69.1
SAMPLE_7      103.3
SAMPLE_8       44.0
SAMPLE_9       83.6
SAMPLE_10      71.1
SAMPLE_11      66.8
SAMPLE_12      96.0
SAMPLE_13      85.5
SAMPLE_14      94.9
SAMPLE_15     132.6
SAMPLE_16      99.4
SAMPLE_17     107.2
SAMPLE_18      83.0
SAMPLE_19     126.2
SAMPLE_20     119.3
SAMPLE_21      65.6
SAMPLE_22      92.8
SAMPLE_23      41.0
SAMPLE_24      74.8
SAMPLE_25     104.6
SAMPLE_26     107.7
SAMPLE_27      76.5
SAMPLE_28      80.2
SAMPLE_29      78.2
SAMPLE_30      87.0
dtype: float64

#### Normalization of data

SAMPLE_3 number of classified reads is 8 to 24 times greater than other samples' reads counts. So that futur analyses not to be biased towards SAMPLE_3, we need to perform some kind of normalization.

Moonstone offers assistance in normalizing your data using many different methods (see [list](https://moonstone.readthedocs.io/en/latest/api_docs/normalization.html))

(To better understand what each normalization method entails, you can watch this [youtube video](https://www.youtube.com/watch?v=UFB993xufUU&t=683s))

In [24]:
from moonstone.normalization.counts.geometric_mean import (
    GeometricMeanNormalization
)

geom_mean_norm = GeometricMeanNormalization(counts_dataframe)     # instantiation
counts_dataframe_normalized = geom_mean_norm.normalized_df

In [25]:
counts_dataframe_normalized.sum()

SAMPLE_1      37.965404
SAMPLE_2      70.253198
SAMPLE_3      97.356079
SAMPLE_4      43.010147
SAMPLE_5      76.953969
SAMPLE_6      39.242644
SAMPLE_7      88.312914
SAMPLE_8      65.842962
SAMPLE_9      69.205156
SAMPLE_10     43.909560
SAMPLE_11     57.492194
SAMPLE_12     66.351561
SAMPLE_13     62.120312
SAMPLE_14     63.759901
SAMPLE_15    117.252168
SAMPLE_16     91.964687
SAMPLE_17     77.238007
SAMPLE_18    124.498432
SAMPLE_19     90.927579
SAMPLE_20    109.840434
SAMPLE_21     44.833819
SAMPLE_22    130.761990
SAMPLE_23     43.861387
SAMPLE_24    128.754973
SAMPLE_25    130.337308
SAMPLE_26     91.235838
SAMPLE_27     79.049005
SAMPLE_28     85.073605
SAMPLE_29     95.690900
SAMPLE_30     75.220596
dtype: float64

### Metadata

In [44]:
import pandas as pd

metadata_file = "moonstone_tuto_metadata_file.csv"
metadata_dataframe = pd.read_csv(metadata_file, sep=",", index_col="SAMPLE_ID")

In [45]:
metadata_dataframe

,SMOKER,GROUP
SAMPLE_ID,,
SAMPLE_1,yes,A
SAMPLE_2,yes,C
SAMPLE_3,no,A
SAMPLE_4,no,C
SAMPLE_5,no,B
SAMPLE_6,no,C
SAMPLE_7,yes,A
SAMPLE_8,no,C
SAMPLE_9,yes,A


In [46]:
pd.crosstab(metadata_dataframe['SMOKER'], metadata_dataframe['GROUP'])

GROUP,A,B,C
SMOKER,,,
no,5,4,5
yes,6,6,4


# Data visualization/exploration

In [28]:
from moonstone.plot.counts import PlotTaxonomyCounts

instance = PlotTaxonomyCounts(counts_dataframe_normalized)

In [29]:
fig1 = instance.plot_most_prevalent_taxa(
    mode="bar",
    mean_threshold=6,
    taxa_number=3,
    taxa_level="species",
    higher_classification=False,               # Set to False remove every rows "xxx(higher taxa)"
    ascending=False,
)

In [30]:
fig2 = instance.plot_most_abundant_taxa(
    mode="boxplot",
    taxa_level="species",
    prevalence_threshold=None,
    average_relative_abundance_threshold=5,
    higher_classification=False,                
    ascending=False,
    output_file="Most_abundant_species.html"        # It's also possible to generate a static image. You just need to change the extension of the file
)

In [47]:
fig3 = instance.plot_sample_composition_most_abundant_taxa(
    taxa_level="species",
    taxa_number=3,                        # the X top species will be represented, the other will be under "Others"
    cluster_samples=True,                 # cluster samples according to their composition in top species/Others (default set to True)
    colors={"Others" : "#d1dae8"},        # set the color of a species (or of "Others")
    color_df=metadata_dataframe["GROUP"], # series or dataframe of metadata to add at the bottom of the graph
    sep_series=metadata_dataframe["SMOKER"].replace({"yes": "smoker", "no": "non smoker"}),
    sep_how="labels",
    output_file=None,
    plotting_options={"layout": {"yaxis_title": "Relative abundance"}}   # all moonstone's plot and graph methods allow you to give your own plotting options
                                                                         # it relies on the fig.update_X({*dictionary*}) methods of plotly
                                                                         # it should be given in a dictionary with X being the first key and then a dictionary of the
                                                                         # parameters to update as you would give the fig.update_X method
)

# Diversity analysis

The Alpha diversity (α-diversity) corresponds to the intra-sample diversity, whereas the Beta diversity (β-diversity) corresponds to the inter-samples diversity.

In [48]:
"""
Alpha-Diversity (at species-level)
"""

from moonstone.analysis.diversity.alpha import ShannonIndex

cdn_species = counts_dataframe_normalized.groupby("species").sum()

alpha_div_instance = ShannonIndex(cdn_species)
alpha_div_instance.diversity_indexes.to_csv(f"ShannonIndex.csv")
grouped_output = alpha_div_instance.analyse_groups(
    metadata_df=metadata_dataframe, group_col="SMOKER",
    stats_test="mann_whitney_u", correction_method="fdr_bh",
    boxpoints="suspectedoutliers", structure_pval="series", sym=False,
    show_pval=False,
)
print(grouped_output["pval"])

Less than 20 observations for group no in data.
Less than 20 observations for group yes in data.


Group1  Group2
no      yes       0.053232
dtype: float64


In [50]:
"""
Beta-Diversity (at species-level)
"""

from moonstone.analysis.diversity.beta import BrayCurtis

cdn_species = counts_dataframe_normalized.groupby("species").sum()

beta_div_instance = BrayCurtis(cdn_species)
beta_div_instance.diversity_indexes.to_csv(f"BrayCurtis.csv")
grouped_output = beta_div_instance.analyse_groups(
    metadata_df=metadata_dataframe, group_col="SMOKER", group_col2="GROUP",
    stats_test="mann_whitney_u", correction_method="fdr_bh",
    boxpoints="suspectedoutliers", structure_pval="series", sym=False,
    show_pval=False,
)
print(grouped_output["pval"])

Less than 20 observations for group no - A in data.
Less than 20 observations for group no - B in data.
Less than 20 observations for group no - C in data.
Less than 20 observations for group yes - A in data.
Less than 20 observations for group yes - B in data.
Less than 20 observations for group yes - C in data.


Group1   Group2 
no - A   no - B     0.233516
         no - C     0.192066
         yes - A    0.610093
         yes - B    0.507999
         yes - C    0.551449
no - B   no - C     0.129870
         yes - A    0.168205
         yes - B    0.129870
         yes - C    0.129870
no - C   yes - A    0.436920
         yes - B    0.373161
         yes - C    0.583394
yes - A  yes - B    0.969851
         yes - C    0.969851
yes - B  yes - C    0.969851
dtype: float64


# Machine-Learning Classification

In [55]:
from moonstone.analysis.classify import ML

ML_instance = ML(counts_dataframe_normalized, metadata_dataframe, "moonstone_tuto_outdir")

In [57]:
# SVM = Support Vector Machine
# ROC = Receiver Operating Characteristic
ML_instance.svm("SMOKER")

ValueError: cannot join with no overlapping index names